In [1]:
import json
import pandas as pd
import numpy as np
import os
import shutil
import pickle
from lightfm.data import Dataset
from scipy.io import mmwrite
from lightfm.cross_validation import random_train_test_split
from hyperopt import fmin, hp, tpe, Trials
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


# 0. 데이터 불러오기

In [2]:
DATA_DIR = "data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

# 데이터 불러들이는 함수
def import_data(data_path=DATA_FILE):
    
    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    reviews = []  # 리뷰 테이블

    for d in data:
        for c in d["category_list"]:
            category = c["category"]
            break

        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [d["id"],d["name"],category,d["address"], u["id"], r["score"], r["reg_time"]]
            )
            # 식당 ID, 식당이름, 카테고리, 작성자 ID, 평점, 시켜먹은 시간


    review_frame = pd.DataFrame(data=reviews, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time"))

    return {"reviews": review_frame}

def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)

def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


In [6]:
data = import_data()
dump_dataframes(data)
data = load_dataframes()

# 평점 데이터 
ratings = data["reviews"]
print("평점 데이터")
ratings

평점 데이터


,store_id,store_name,category,address,user_id,score,reg_time
0,15,써리힐,호주레스토랑,부산광역시 부산진구 전포동 229-13번지 2층,68632,5,1970-01-01 00:00:00
1,18,진삼미 샌드위치,샌드위치,전라북도 전주시 완산구 효자동1가 652 상산고등학교앞,389728,5,1970-01-01 00:00:00
2,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00
3,37,007식당,굴비정식,전라남도 영광군 법성면 법성리 1148-6,774353,2,1970-01-01 00:00:00
4,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47
...,...,...,...,...,...,...,...
91393,360499,지구촌한우생고기,한우,부산광역시 해운대구 중2동 503-1,17371,5,1970-01-01 00:00:00
91394,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00
91395,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00
91396,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00


# 1. 제외할 카테고리 데이터 삭제하기

**1) 빈카테고리 <br>
2) 카페, cafe, 커피, coffee,  더치, 쥬스, 주스, 버블티, 스무디,<br>
3) 빵, 수플레, 샌드위치,  브레드, 마카롱,<br>
4) 아이스바, 라즈베리,<br>
5) 레스토랑, 식당, 그리스,<br>
6) 주점, 맥주, 와인, 술집, 이자카야, pub,  포차, 칵테일,**

In [16]:
ratings = data["reviews"]
ratings

,store_id,store_name,category,address,user_id,score,reg_time
0,15,써리힐,호주레스토랑,부산광역시 부산진구 전포동 229-13번지 2층,68632,5,1970-01-01 00:00:00
1,18,진삼미 샌드위치,샌드위치,전라북도 전주시 완산구 효자동1가 652 상산고등학교앞,389728,5,1970-01-01 00:00:00
2,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00
3,37,007식당,굴비정식,전라남도 영광군 법성면 법성리 1148-6,774353,2,1970-01-01 00:00:00
4,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47
...,...,...,...,...,...,...,...
91393,360499,지구촌한우생고기,한우,부산광역시 해운대구 중2동 503-1,17371,5,1970-01-01 00:00:00
91394,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00
91395,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00
91396,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00


In [17]:
categorys = ["카페","cafe","커피","coffee","더치","쥬스","주스","버블티","스무디","빵","수플레",
             "샌드위치","브레드","마카롱","아이스바","라즈베리","레스토랑","식당","그리스","주점","맥주","와인",
            "술집","이자카야","pub","포차","칵테일"]

In [18]:
for i in range(ratings.shape[0]):
    if ratings['category'][i] == "" :
        print(ratings['store_id'][i],"번 가게가 카테고리가 없어서 삭제하겠습니다.")
        ratings = ratings.drop(i,0)

1660 번 가게가 카테고리가 없어서 삭제하겠습니다.
12489 번 가게가 카테고리가 없어서 삭제하겠습니다.
15058 번 가게가 카테고리가 없어서 삭제하겠습니다.
18118 번 가게가 카테고리가 없어서 삭제하겠습니다.
18787 번 가게가 카테고리가 없어서 삭제하겠습니다.
27619 번 가게가 카테고리가 없어서 삭제하겠습니다.
30700 번 가게가 카테고리가 없어서 삭제하겠습니다.
34116 번 가게가 카테고리가 없어서 삭제하겠습니다.
34931 번 가게가 카테고리가 없어서 삭제하겠습니다.
35551 번 가게가 카테고리가 없어서 삭제하겠습니다.
46712 번 가게가 카테고리가 없어서 삭제하겠습니다.
47084 번 가게가 카테고리가 없어서 삭제하겠습니다.
47084 번 가게가 카테고리가 없어서 삭제하겠습니다.
47084 번 가게가 카테고리가 없어서 삭제하겠습니다.
53544 번 가게가 카테고리가 없어서 삭제하겠습니다.
59486 번 가게가 카테고리가 없어서 삭제하겠습니다.
65398 번 가게가 카테고리가 없어서 삭제하겠습니다.
65398 번 가게가 카테고리가 없어서 삭제하겠습니다.
68341 번 가게가 카테고리가 없어서 삭제하겠습니다.
72325 번 가게가 카테고리가 없어서 삭제하겠습니다.
74724 번 가게가 카테고리가 없어서 삭제하겠습니다.
82491 번 가게가 카테고리가 없어서 삭제하겠습니다.
83123 번 가게가 카테고리가 없어서 삭제하겠습니다.
86786 번 가게가 카테고리가 없어서 삭제하겠습니다.
99250 번 가게가 카테고리가 없어서 삭제하겠습니다.
102181 번 가게가 카테고리가 없어서 삭제하겠습니다.
104085 번 가게가 카테고리가 없어서 삭제하겠습니다.
107023 번 가게가 카테고리가 없어서 삭제하겠습니다.
111927 번 가게가 카테고리가 없어서 삭제하겠습니다.
115146 번 가게가 카테고리가 없어서 삭제하겠습니다.
119800 번 가게가 카테고리가 없어서 삭제하겠습니다.
119800 번 가게가 카테고리가 없어서 삭제하겠습니다.
12

In [19]:
# 줄한번 맞추기. index컬럼 삭제.
ratings = ratings.reset_index() 
del ratings['index']
ratings

,store_id,store_name,category,address,user_id,score,reg_time
0,15,써리힐,호주레스토랑,부산광역시 부산진구 전포동 229-13번지 2층,68632,5,1970-01-01 00:00:00
1,18,진삼미 샌드위치,샌드위치,전라북도 전주시 완산구 효자동1가 652 상산고등학교앞,389728,5,1970-01-01 00:00:00
2,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00
3,37,007식당,굴비정식,전라남도 영광군 법성면 법성리 1148-6,774353,2,1970-01-01 00:00:00
4,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47
...,...,...,...,...,...,...,...
91254,360499,지구촌한우생고기,한우,부산광역시 해운대구 중2동 503-1,17371,5,1970-01-01 00:00:00
91255,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00
91256,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00
91257,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00


In [20]:
# 제외할 카테고리를 하나씩 차례로 검사한다.
for category in categorys:
    for i in range(ratings.shape[0]):
        if category in ratings['category'][i]:
            print(ratings['store_id'][i],"번 가게가 카테고리가", ratings['category'][i] ," 라서 삭제하겠습니다.")
            ratings = ratings.drop(i,0)
    
    # 줄한번 맞추기. index컬럼 삭제.
    ratings = ratings.reset_index() 
    del ratings['index']

91 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
123 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
184 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
239 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
239 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
239 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
316 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
316 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
328 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
435 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
532 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
534 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
540 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
550 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
578 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
601 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
748 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
798 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
818 번 가게가 카테고리가 베이커리카페  라서 삭제하겠습니다.
818 번 가게가 카테고리가 베이커리카페  라서 삭제하겠습니다.
818 번 가게가 카테고리가 베이커리카페  라서 삭제하겠습니다.
856 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
965 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
972 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
972 번 가게가 카테고리가 한옥카페  라서

12109 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13332 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13353 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13353 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13384 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13502 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13502 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
13503 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
13505 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
15766 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
15766 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
15766 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
15766 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
15873 번 가게가 카테고리가 이색카페  라서 삭제하겠습니다.
16468 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16468 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16468 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16468 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16550 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16550 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16550 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16551 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
16600 번 가게가 카테고리가 카페  라서 삭제하겠습

45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
45753 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
46283 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
47062 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
47694 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
47990 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
47990 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
48104 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
48427 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
48427 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
48731 번 가게가 카테고리가 전원카페  라서 삭제하겠습니다.
49106 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
49106 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
49106 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
49106 번 가게가 카테고리가 초콜릿카페  라서 삭제하겠습니다.
50346 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
53362 번 가게가 카테고리가 로스터리카페  라서 삭제하겠습니다.
54193 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
54193 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
54193 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
54270 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
54275 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
54281 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
54290 번 가게가 카테고리가 플라워카페  라서 삭제하겠습니다.
54300 번 가게가 카테고리가 

76245 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76245 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76245 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76245 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76400 번 가게가 카테고리가 갤러리카페  라서 삭제하겠습니다.
76400 번 가게가 카테고리가 갤러리카페  라서 삭제하겠습니다.
76428 번 가게가 카테고리가 애견카페  라서 삭제하겠습니다.
76482 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76793 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
76925 번 가게가 카테고리가 베이커리카페  라서 삭제하겠습니다.
77093 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
77289 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
77421 번 가게가 카테고리가 한옥카페  라서 삭제하겠습니다.
77580 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
77878 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
77878 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
78089 번 가게가 카테고리가 만화카페  라서 삭제하겠습니다.
78187 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
78187 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
78215 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
78215 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
78433 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
78434 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
78520 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
78520 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
78688 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
78691 번 가게가 카테고리가 디저트카페  라서 삭제하겠습니다.
78791 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
78

95202 번 가게가 카테고리가 이색카페  라서 삭제하겠습니다.
95202 번 가게가 카테고리가 이색카페  라서 삭제하겠습니다.
95205 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95214 번 가게가 카테고리가 브런치카페  라서 삭제하겠습니다.
95293 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95454 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95747 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95747 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95747 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95798 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95873 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95873 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
95883 번 가게가 카테고리가 갤러리카페  라서 삭제하겠습니다.
96008 번 가게가 카테고리가 북카페  라서 삭제하겠습니다.
96056 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
96279 번 가게가 카테고리가 애견카페  라서 삭제하겠습니다.
96423 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
96475 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97020 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97020 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97020 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97021 번 가게가 카테고리가 식물카페  라서 삭제하겠습니다.
97074 번 가게가 카테고리가 드로잉카페  라서 삭제하겠습니다.
97075 번 가게가 카테고리가 드로잉카페  라서 삭제하겠습니다.
97089 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97089 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
97089 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
100958 번 가게가 카테고리가 카페  라서 삭제하겠습니다.
100958 번 가게가 카테고리가 카페  라서 

...

294268 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
295508 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
295508 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
295953 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
297337 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
297662 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
298494 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
299173 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
299175 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
301082 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
302316 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
303103 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
303320 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
303541 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
303541 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
305488 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
305823 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
307983 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
311114 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
311115 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
314046 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
316227 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
316359 번 가게가 카테고리가 선술집  라서 삭제하겠습니다.
316359 번 가게가 카테고리가 선술집  라서 삭제하겠습니다.
316530 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
317285 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
317285 번 가게가 카테고리가 술집  라서 삭제하겠습니다.
318394 번 가게가 카테고리가 선술집  라서 삭제하겠습니다.
318394 번 가게가 카테고리

101452 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
101487 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
101487 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
101487 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
101487 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
101488 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
119428 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121212 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121574 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121574 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121853 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121935 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
121939 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
122168 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
122265 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
122266 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
122498 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
123691 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
123751 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
125202 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
125202 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
125202 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
130645 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
130645 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
132348 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
134321 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
137081 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
1

290361 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
290361 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
290597 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
290597 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
293553 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
293554 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
293564 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
293564 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
293596 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294449 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294449 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294449 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294454 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294533 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
294533 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
295811 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
297422 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299092 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299183 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299431 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299905 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299905 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299933 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
299933 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
303064 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
303297 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
303451 번 가게가 카테고리가 이자카야  라서 삭제하겠습니다.
3

225437 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
227845 번 가게가 카테고리가 실내포차  라서 삭제하겠습니다.
227845 번 가게가 카테고리가 실내포차  라서 삭제하겠습니다.
238325 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
238632 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
244214 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
244214 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247370 번 가게가 카테고리가 해물포차  라서 삭제하겠습니다.
247747 번 가게가 카테고리가 셀프포차  라서 삭제하겠습니다.
250425 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
250425 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
253281 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
260678 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
263436 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
263436 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
265165 번 가게가 카테고리가 중식포차  라서 삭제하겠습니다.
265165 번 가게가 카테고리가 중식포차  라서 삭제하겠습니다.
265165 번 가게가 카테고리가 중식포차  라서 삭제하겠습니다.
265165 번 가게가 카테고리가 중식포차  라서 삭제하겠습니다.
268413 번 가게가 카테고리가 퓨전포차  라서 삭제하겠습니다.
274381 번 가게가 카테고리가 포차  라서 삭제하겠습니다.
281321 번 가게가 카테고리가 포차  라서

317583 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
330801 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
330801 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
331349 번 가게가 카테고리가 칵테일  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
342366 번 가게가 카테고리가 칵테일바  라서 삭제하겠습니다.
348033 번 가게가 카테고리가 칵테일  라서 삭제하겠습니다.
357015 번 가게가 카테고리가 칵테일  라서 삭제하겠습니다.
360475 번 가게가 카테고리가 칵테일  라서 삭제하겠습니다.


In [21]:
ratings

,store_id,store_name,category,address,user_id,score,reg_time
0,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00
1,37,007식당,굴비정식,전라남도 영광군 법성면 법성리 1148-6,774353,2,1970-01-01 00:00:00
2,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47
3,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,388883,5,1970-01-01 00:00:00
4,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,180541,4,1970-01-01 00:00:00
...,...,...,...,...,...,...,...
79747,360499,지구촌한우생고기,한우,부산광역시 해운대구 중2동 503-1,17371,5,1970-01-01 00:00:00
79748,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00
79749,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00
79750,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00


# 2. 시간대별로 day_type 설정하기

**새로운 컬럼 day_type 추가**

In [22]:
ratings=ratings.assign(day_type="")

**reg_time을 datetime형으로 바꾸기**

In [24]:
ratings['reg_time'] = pd.to_datetime(ratings['reg_time'])

In [26]:
for i in range(ratings.shape[0]):
    if int(ratings['reg_time'][i].hour) >= 5 and  int(ratings['reg_time'][i].hour) < 11 :
        ratings['day_type'][i] = 'morning'
    elif int(ratings['reg_time'][i].hour) >= 11 and  int(ratings['reg_time'][i].hour) < 17 :
        ratings['day_type'][i] = 'lunch'
    elif int(ratings['reg_time'][i].hour) >= 17 and  int(ratings['reg_time'][i].hour) < 23 :
        ratings['day_type'][i] = 'dinner'
    else :
        ratings['day_type'][i] = 'night'

<ipython-input-26-f943d36e4dfa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['day_type'][i] = 'night'
<ipython-input-26-f943d36e4dfa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['day_type'][i] = 'dinner'
<ipython-input-26-f943d36e4dfa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['day_type'][i] = 'morning'
<ipython-input-26-f943d36e4dfa>:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [27]:
ratings

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00,night
1,37,007식당,굴비정식,전라남도 영광군 법성면 법성리 1148-6,774353,2,1970-01-01 00:00:00,night
2,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47,dinner
3,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,388883,5,1970-01-01 00:00:00,night
4,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,180541,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
79747,360499,지구촌한우생고기,한우,부산광역시 해운대구 중2동 503-1,17371,5,1970-01-01 00:00:00,night
79748,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00,night
79749,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00,night
79750,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00,night


# 3. 지역별로 데이터 나누기

In [33]:
Seoul = [] #서울특별시
Busan = [] #부산광역시 
Daegu = [] #대구광역시
Inchen = [] # 인천광역시
Gwangju = [] #광주광역시
Daejeon = [] # 대전광역시 
Ulsan = [] #울산광역시
Sejong = [] # 세종특별자치시 
Gyeonggi = [] # 경기도 
Gangwon = [] # 강원도
Chungbuk = [] # 충청북도 
Chungnam = [] # 충청남도 
Jeollabuk = [] # 전라북도 
Jeollanam = [] # 전라남도 
Gyeongbuk = [] # 경상북도
Gyeongnam = [] # 경상남도
Jeju = [] #제주도 

In [34]:
Seoul = []
for i in range(ratings.shape[0]):
    if '서울' in ratings['address'][i].split(" ")[0]:
        Seoul.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Seoul_frame = pd.DataFrame(data=Seoul, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Seoul_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3,2019-03-15 22:16:47,dinner
1,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,388883,5,1970-01-01 00:00:00,night
2,49,03수제비칼국수,수제비,서울특별시 서초구 방배동 768-2,180541,4,1970-01-01 00:00:00,night
3,110,100년설렁탕,설렁탕,서울특별시 성북구 삼선동2가 38,64301,5,2019-02-26 22:15:20,dinner
4,129,100억곱창,치즈곱창,서울특별시 강서구 화곡6동 986-22,148720,5,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
35129,360492,지구촌,루프탑,서울특별시 용산구 이태원동 130-43,574786,4,1970-01-01 00:00:00,night
35130,360493,지구촌 가정식,미고랭,서울특별시 마포구 연남동 228-4,419534,5,2018-01-23 09:35:11,morning
35131,360493,지구촌 가정식,미고랭,서울특별시 마포구 연남동 228-4,43793,4,2018-01-27 15:55:35,lunch
35132,360493,지구촌 가정식,미고랭,서울특별시 마포구 연남동 228-4,5239,3,2018-03-19 02:26:09,night


In [35]:
Busan=[]
for i in range(ratings.shape[0]):
    if '부산' in ratings['address'][i].split(" ")[0]:
        Busan.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Busan_frame = pd.DataFrame(data=Busan, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Busan_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,555,17테이블,파스타,부산광역시 금정구 장전동 302-13,22782,5,2017-09-02 08:52:15,morning
1,555,17테이블,파스타,부산광역시 금정구 장전동 302-13,257609,3,1970-01-01 00:00:00,night
2,555,17테이블,파스타,부산광역시 금정구 장전동 302-13,357677,5,1970-01-01 00:00:00,night
3,585,18번완당집,완당,부산광역시 사하구 하단동 526-6,39188,3,2019-04-01 21:04:47,dinner
4,585,18번완당집,완당,부산광역시 사하구 하단동 526-6,150132,3,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
4079,359235,중앙양고기,양꼬치,부산광역시 중구 광복동1가 19-1 2층,26298,5,1970-01-01 00:00:00,night
4080,359235,중앙양고기,양꼬치,부산광역시 중구 광복동1가 19-1 2층,131381,4,1970-01-01 00:00:00,night
4081,359578,중화요리궁,중화요리,부산광역시 동래구 명륜동 541-2,861280,3,1970-01-01 00:00:00,night
4082,360393,즐거울락,양꼬지,부산광역시 사상구 주례2동 166-159,837952,4,2017-04-11 03:29:50,night


In [36]:
Daegu=[]
for i in range(ratings.shape[0]):
    if '대구' in ratings['address'][i].split(" ")[0]:
        Daegu.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Daegu_frame = pd.DataFrame(data=Daegu, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Daegu_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,60,053키친,리조또,대구광역시 중구 완전동 5-15,450355,4,2017-07-01 20:12:33,dinner
1,395,12키친,파인다이닝,대구광역시 수성구 상동 408-3,88418,4,1970-01-01 00:00:00,night
2,430,13월의식당,뼈다귀해장국,대구광역시 수성구 두산동 88-4,666389,3,2018-01-08 10:32:07,morning
3,517,161 커피 스튜디오,스튜,대구광역시 달성군 현풍면 원교리 161,213776,3,1970-01-01 00:00:00,night
4,615,1943,치킨,대구광역시 중구 삼덕동1가 57 2층,149529,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
2066,359506,중화반점,야끼우동,대구광역시 중구 남일동 92,745243,3,1970-01-01 00:00:00,night
2067,359506,중화반점,야끼우동,대구광역시 중구 남일동 92,315209,5,1970-01-01 00:00:00,night
2068,359506,중화반점,야끼우동,대구광역시 중구 남일동 92,861280,3,1970-01-01 00:00:00,night
2069,360468,지구당,규동,대구광역시 중구 동성로2가 67-4,866067,5,1970-01-01 00:00:00,night


In [37]:
Inchen=[]
for i in range(ratings.shape[0]):
    if '인천' in ratings['address'][i].split(" ")[0]:
        Inchen.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Inchen_frame = pd.DataFrame(data=Inchen, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Inchen_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,419,136생고기,소고기무한리필,인천광역시 계양구 작전동 136,115004,3,2018-04-24 13:27:27,lunch
1,752,1988고기불패,돼지고기전문,인천광역시 계양구 계산동 1082-6,721423,1,1970-01-01 00:00:00,night
2,790,1989고깃집,고기집,인천광역시 연수구 송도동 190-2,306161,5,1970-01-01 00:00:00,night
3,1143,21세기바다,랍스타,인천시 남동구 구월동 1136-4,15142,5,1970-01-01 00:00:00,night
4,1494,24시전주콩나물국밥,콩나물국밥,인천광역시 연수구 청학동 496-5,664880,3,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
2879,359434,중화루,유니짜장,인천시 중구 중앙동4가 7-1,259677,3,1970-01-01 00:00:00,night
2880,359434,중화루,유니짜장,인천시 중구 중앙동4가 7-1,155100,4,1970-01-01 00:00:00,night
2881,359434,중화루,유니짜장,인천시 중구 중앙동4가 7-1,664880,5,1970-01-01 00:00:00,night
2882,359434,중화루,유니짜장,인천시 중구 중앙동4가 7-1,510279,4,1970-01-01 00:00:00,night


In [38]:
Gwangju=[]
for i in range(ratings.shape[0]):
    if '광주' in ratings['address'][i].split(" ")[0]:
        Gwangju.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gwangju_frame = pd.DataFrame(data=Gwangju, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Gwangju_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,133,100족발삶는집,족발,광주광역시 광산구 신창동 1256-1,774353,4,1970-01-01 00:00:00,night
1,133,100족발삶는집,족발,광주광역시 광산구 신창동 1256-1,49902,5,1970-01-01 00:00:00,night
2,542,172후라이드라이스,철판볶음밥,광주광역시 서구 화정동 1302,401452,5,2018-10-21 20:58:03,dinner
3,637,1960청원모밀,모밀,광주광역시 서구 치평동 1278-2,714613,4,2016-07-06 12:26:30,lunch
4,637,1960청원모밀,모밀,광주광역시 서구 치평동 1278-2,514320,5,2017-01-01 15:39:37,lunch
...,...,...,...,...,...,...,...,...
999,358448,중,짬뽕,광주광역시 북구 문흥동 741-32,85130,4,1970-01-01 00:00:00,night
1000,358796,중독 뒷고기정육점빵,뒷고기,광주광역시 북구 용봉동 1387-2,72073,5,2018-05-16 09:32:32,morning
1001,358954,중산,중화요리,광주광역시 서구 광천동 49-1,285575,3,1970-01-01 00:00:00,night
1002,359050,중앙닭갈비,닭갈비,광주광역시 동구 황금동 19-1,81330,3,1970-01-01 00:00:00,night


In [39]:
Daejeon=[]
for i in range(ratings.shape[0]):
    if '대전' in ratings['address'][i].split(" ")[0]:
        Daejeon.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Daejeon_frame = pd.DataFrame(data=Daejeon, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Daejeon_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,248,111 7meal,함박스테이크,대전광역시 유성구 어은동 111-7 1층,583035,4,2019-03-01 00:24:50,night
1,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,314095,5,1970-01-01 00:00:00,night
2,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,328037,5,1970-01-01 00:00:00,night
3,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,190973,4,1970-01-01 00:00:00,night
4,737,1987루프탑,루프탑라운지,대전광역시 서구 탄방동 740,421128,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
2319,358026,죽이야기,죽집,대전광역시 서구 둔산1동 1412,151946,4,1970-01-01 00:00:00,night
2320,358161,준구네껍데기,껍데기,대전광역시 중구 유천2동 196,96614,1,1970-01-01 00:00:00,night
2321,358352,준호네춘천숯불닭갈비,숯불닭갈비,대전광역시 유성구 전민동 336-4,362995,3,1970-01-01 00:00:00,night
2322,358949,중부회24시수산시장,대하구이,대전광역시 동구 홍도동 32,194919,5,1970-01-01 00:00:00,night


In [40]:
Ulsan=[]
for i in range(ratings.shape[0]):
    if '울산' in ratings['address'][i].split(" ")[0]:
        Ulsan.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Ulsan_frame = pd.DataFrame(data=Ulsan, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Ulsan_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,56,04위로,차돌박이짬뽕,울산광역시 남구 무거동 628-26,946187,4,2016-07-07 20:51:23,dinner
1,56,04위로,차돌박이짬뽕,울산광역시 남구 무거동 628-26,319105,3,2017-08-22 19:52:50,dinner
2,56,04위로,차돌박이짬뽕,울산광역시 남구 무거동 628-26,183349,5,2016-07-07 20:51:23,dinner
3,1154,225토마토스트릿,스테이크,울산광역시 중구 태화동 471-16,319105,4,2017-08-27 20:58:59,dinner
4,1154,225토마토스트릿,스테이크,울산광역시 중구 태화동 471-16,395154,3,2018-12-19 21:29:47,dinner
...,...,...,...,...,...,...,...,...
749,356898,주산지,명태찜,울산광역시 울주군 범서읍 구영리 394-10,327367,5,1970-01-01 00:00:00,night
750,358454,중경삼림,팔보채,울산광역시 울주군 범서읍 천상리 329-1,399595,5,2019-01-26 04:04:34,night
751,358672,중국성,중국집,울산광역시 울주군 온양읍 대안리 187-4,397051,4,1970-01-01 00:00:00,night
752,358956,중산뒷고기막창,삼겹살,울산광역시 북구 중산동 1287-3,286165,3,1970-01-01 00:00:00,night


In [41]:
Sejong=[]
for i in range(ratings.shape[0]):
    if '세종' in ratings['address'][i].split(" ")[0]:
        Sejong.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Sejong_frame = pd.DataFrame(data=Sejong, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Sejong_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,411,135튀김집,튀김,세종특별자치시 새롬동 341-74,148970,5,1970-01-01 00:00:00,night
1,559,180도씨,수제튀김,세종특별자치시 고운동 1721,389025,4,1970-01-01 00:00:00,night
2,699,1980황가원,석갈비,세종특별자치시 어진동 547,201760,4,1970-01-01 00:00:00,night
3,1056,2005진돈가스,돈가스,세종특별자치시 조치원읍 교리 6-9,826730,4,2016-08-28 18:43:31,dinner
4,1235,24시 한뚝 수육국밥,수육국밥,세종특별자치시 도담동 660 한신휴시티,389025,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
272,356677,주문리동치미막국수,동치미,세종특별자치시 연기면 연기리 508,151946,4,1970-01-01 00:00:00,night
273,357657,죽림,한우,세종시 금남면 축산리 147-2,126107,5,1970-01-01 00:00:00,night
274,358941,중부수산,횟집,세종특별자치시 종촌동 674,151946,4,1970-01-01 00:00:00,night
275,359519,중화반점,짬뽕,세종특별자치시 새롬동 301 10단지상가동,215624,5,1970-01-01 00:00:00,night


In [42]:
Gyeonggi=[]
for i in range(ratings.shape[0]):
    if '경기' in ratings['address'][i].split(" ")[0]:
        Gyeonggi.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gyeonggi_frame = pd.DataFrame(data=Gyeonggi, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Gyeonggi_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4,1970-01-01 00:00:00,night
1,58,04하우스,피자,경기도 부천시 원미구 상동 411,31285,5,1970-01-01 00:00:00,night
2,58,04하우스,피자,경기도 부천시 원미구 상동 411,6757,5,1970-01-01 00:00:00,night
3,68,080부대찌개,수제소세지,경기도 고양시 덕양구 화정동 975-2,137715,5,2017-11-07 20:14:36,dinner
4,68,080부대찌개,수제소세지,경기도 고양시 덕양구 화정동 975-2,290317,4,2017-11-09 21:14:50,dinner
...,...,...,...,...,...,...,...,...
15175,359400,중화가정,짬뽕,경기도 시흥시 조남동 산15-1,243883,4,1970-01-01 00:00:00,night
15176,360505,지군포차,국물닭발,경기도 부천시 역곡동 77-12,198050,4,1970-01-01 00:00:00,night
15177,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,190766,5,1970-01-01 00:00:00,night
15178,360514,지글,고기뷔페,경기도 성남시 분당구 정자동 162-2,201564,4,1970-01-01 00:00:00,night


In [43]:
Gangwon=[]
for i in range(ratings.shape[0]):
    if '강원' in ratings['address'][i].split(" ")[0]:
        Gangwon.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gangwon_frame = pd.DataFrame(data=Gangwon, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Gangwon_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,86,1.5닭갈비,닭갈비,강원도 춘천시 후평3동 801-13,116795,1,1970-01-01 00:00:00,night
1,86,1.5닭갈비,닭갈비,강원도 춘천시 후평3동 801-13,129160,4,1970-01-01 00:00:00,night
2,86,1.5닭갈비,닭갈비,강원도 춘천시 후평3동 801-13,592429,4,1970-01-01 00:00:00,night
3,86,1.5닭갈비,닭갈비,강원도 춘천시 후평3동 801-13,283542,4,1970-01-01 00:00:00,night
4,86,1.5닭갈비,닭갈비,강원도 춘천시 후평3동 801-13,53458,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
3121,359051,중앙닭강정,닭강정,강원도 속초시 중앙동 471-4 속초중앙시장,697977,3,1970-01-01 00:00:00,night
3122,359499,중화문,중식,강원도 원주시 단계동 407-12,752578,3,2017-12-15 18:08:20,dinner
3123,359639,중화짬뽕빵,불짬뽕,강원도 강릉시 성남동 51-106,41555,4,2019-01-29 22:32:16,dinner
3124,359639,중화짬뽕빵,불짬뽕,강원도 강릉시 성남동 51-106,245746,4,2019-05-11 00:52:22,night


In [44]:
Chungbuk = []
for i in range(ratings.shape[0]):
    if '충북' in ratings['address'][i].split(" ")[0]:
        Chungbuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Chungbuk_frame = pd.DataFrame(data=Chungbuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Chungbuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,23284,4,2018-05-20 20:43:49,dinner
1,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,68122,1,2018-08-02 19:10:51,dinner
2,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,3141,5,2018-10-27 12:01:08,lunch
3,38198,구박사소곱창구이,소곱창구이,충북 청주시 흥덕구 봉명2동 2850,253222,5,2018-09-19 18:01:49,dinner
4,59643,나나네생고기,분식,충북 진천군 이월면 송림리 448-41,420421,5,2019-04-15 23:14:18,night
5,73788,다낭,베트남쌀국수,충북 청주시 흥덕구 오송읍 연제리 719,200863,4,1970-01-01 00:00:00,night
6,73788,다낭,베트남쌀국수,충북 청주시 흥덕구 오송읍 연제리 719,27703,4,1970-01-01 00:00:00,night
7,81951,대게랑,킹크랩,충북 청주시 청원구 율량동2135,395537,5,1970-01-01 00:00:00,night
8,114088,또또와닭발,닭발,충북 청주시 청원구 우암동 843-3,832822,1,2017-04-22 16:03:54,lunch
9,125685,리코,스테이크,충북 영동군 영동읍 매천리 523-2,636781,4,1970-01-01 00:00:00,night


In [46]:
for i in range(ratings.shape[0]):
    if '충청북' in ratings['address'][i].split(" ")[0]:
        Chungbuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Chungbuk_frame = pd.DataFrame(data=Chungbuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Chungbuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,23284,4,2018-05-20 20:43:49,dinner
1,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,68122,1,2018-08-02 19:10:51,dinner
2,15129,가찬,중국집,충북 음성군 맹동면 두성리 1266,3141,5,2018-10-27 12:01:08,lunch
3,38198,구박사소곱창구이,소곱창구이,충북 청주시 흥덕구 봉명2동 2850,253222,5,2018-09-19 18:01:49,dinner
4,59643,나나네생고기,분식,충북 진천군 이월면 송림리 448-41,420421,5,2019-04-15 23:14:18,night
...,...,...,...,...,...,...,...,...
1175,359283,중앙탑막국수,메밀치킨,충청북도 충주시 중앙탑면 탑평리 41,75341,3,1970-01-01 00:00:00,night
1176,359346,중원순대,순대국밥,충청북도 충주시 충의동 101-48,255093,4,1970-01-01 00:00:00,night
1177,360335,즐거운나의집돌솥밥,돌솥밥,충청북도 청주시 상당구 금천동 402,9852,5,2017-12-18 20:11:24,dinner
1178,360335,즐거운나의집돌솥밥,돌솥밥,충청북도 청주시 상당구 금천동 402,247010,4,2018-01-23 13:19:59,lunch


In [47]:
Chungnam=[]
for i in range(ratings.shape[0]):
    if '충남' in ratings['address'][i].split(" ")[0]:
        Chungnam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Chungnam_frame = pd.DataFrame(data=Chungnam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Chungnam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,9344,LA 떡볶이,떡볶이,충남 아산시 배방읍 장재리 1482,307295,5,1970-01-01 00:00:00,night
1,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,385143,4,1970-01-01 00:00:00,night
2,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,727149,4,1970-01-01 00:00:00,night
3,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,155982,4,1970-01-01 00:00:00,night
4,75049,다미횟집,전복구이,충남 태안군 안면읍 승언리 1329,871664,2,1970-01-01 00:00:00,night
5,75049,다미횟집,전복구이,충남 태안군 안면읍 승언리 1329,536316,1,1970-01-01 00:00:00,night
6,86072,대양횟집,게국지,충남 태안군 안면읍 승언리 1330,946279,1,1970-01-01 00:00:00,night
7,126718,마늘떡볶이 TTEOK BOKKI,마늘떡볶이,충남 천안시 서북구 쌍용동 415-1,8731,4,1970-01-01 00:00:00,night
8,127438,마라유혹,마라탕,충남 천안시 동남구 대흥동 60-1,8731,4,1970-01-01 00:00:00,night
9,137907,맛찬들왕소금구이,삼겹살,충남 천안시 서북구 두정동 808,587319,4,1970-01-01 00:00:00,night


In [48]:
for i in range(ratings.shape[0]):
    if '충청남' in ratings['address'][i].split(" ")[0]:
        Chungnam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Chungnam_frame = pd.DataFrame(data=Chungnam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Chungnam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,9344,LA 떡볶이,떡볶이,충남 아산시 배방읍 장재리 1482,307295,5,1970-01-01 00:00:00,night
1,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,385143,4,1970-01-01 00:00:00,night
2,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,727149,4,1970-01-01 00:00:00,night
3,26309,고기원칙,고기집,충남 천안시 서북구 불당동 1623,155982,4,1970-01-01 00:00:00,night
4,75049,다미횟집,전복구이,충남 태안군 안면읍 승언리 1329,871664,2,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
1657,359116,중앙분식,비빔만두,충청남도 공주시 반죽동 268-1,68559,4,1970-01-01 00:00:00,night
1658,359120,중앙산채명가(중앙식당),더덕구이,충청남도 예산군 덕산면 사천리 25-15,804534,4,2018-05-20 14:06:34,lunch
1659,359120,중앙산채명가(중앙식당),더덕구이,충청남도 예산군 덕산면 사천리 25-15,73510,3,1970-01-01 00:00:00,night
1660,359120,중앙산채명가(중앙식당),더덕구이,충청남도 예산군 덕산면 사천리 25-15,290833,3,1970-01-01 00:00:00,night


In [49]:
Jeollabuk = [] # 전라북도 
for i in range(ratings.shape[0]):
    if '전북' in ratings['address'][i].split(" ")[0]:
        Jeollabuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Jeollabuk_frame = pd.DataFrame(data=Jeollabuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Jeollabuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,8304,EIGHT FLOOR,루프탑,전북 전주시 완산구 효자동3가 1694-2 8층,389728,5,1970-01-01 00:00:00,night
1,17436,감탄떡볶이,분식,전북 전주시 완산구 평화동2가,389728,4,1970-01-01 00:00:00,night
2,22313,건강식당,아귀찜,전북 전주시 완산구 효자동1가 218-2,328029,5,1970-01-01 00:00:00,night
3,29016,고씨네,카레,전북 전주시 덕진구 덕진동1가 1261-4,222032,5,1970-01-01 00:00:00,night
4,125912,리틀하노이,베트남음식,전북 전주시 덕진구 덕진동2가 693-4,397331,4,1970-01-01 00:00:00,night
5,140321,맨스플레이트,오믈렛,전북 전주시 완산구 효자동1가 453-1,191568,4,2019-05-09 14:50:52,lunch
6,155185,물망초,파스타,전북 전주시 완산구 다가동3가 23-2,191568,2,1970-01-01 00:00:00,night
7,181544,복돼지감자탕,감자탕,전북 전주시 완산구 효자동3가 1529-3,417388,3,1970-01-01 00:00:00,night
8,181544,복돼지감자탕,감자탕,전북 전주시 완산구 효자동3가 1529-3,169910,5,1970-01-01 00:00:00,night
9,188095,부뚜막,어죽,전북 진안군 안천면 신괴리 907-2,153040,5,1970-01-01 00:00:00,night


In [50]:
for i in range(ratings.shape[0]):
    if '전라북' in ratings['address'][i].split(" ")[0]:
        Jeollabuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Jeollabuk_frame = pd.DataFrame(data=Jeollabuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Jeollabuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,8304,EIGHT FLOOR,루프탑,전북 전주시 완산구 효자동3가 1694-2 8층,389728,5,1970-01-01 00:00:00,night
1,17436,감탄떡볶이,분식,전북 전주시 완산구 평화동2가,389728,4,1970-01-01 00:00:00,night
2,22313,건강식당,아귀찜,전북 전주시 완산구 효자동1가 218-2,328029,5,1970-01-01 00:00:00,night
3,29016,고씨네,카레,전북 전주시 덕진구 덕진동1가 1261-4,222032,5,1970-01-01 00:00:00,night
4,125912,리틀하노이,베트남음식,전북 전주시 덕진구 덕진동2가 693-4,397331,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
1625,358972,중수원,중국집,전라북도 김제시 요촌동 201-19,937152,5,1970-01-01 00:00:00,night
1626,359191,중앙식당,아나고탕,전라북도 군산시 금암동 3-6,136221,2,1970-01-01 00:00:00,night
1627,359635,중화정,양꼬치,전라북도 전주시 완산구 중화산동2가 780-1,113378,5,1970-01-01 00:00:00,night
1628,359637,중화정,양꼬치,전라북도 전주시 완산구 평화동1가 741-3,45387,5,1970-01-01 00:00:00,night


In [51]:
Jeollanam = [] # 전라남도 
for i in range(ratings.shape[0]):
    if '전남' in ratings['address'][i].split(" ")[0]:
        Jeollanam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Jeollanam_frame = pd.DataFrame(data=Jeollanam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Jeollanam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,5923,BBQ치킨,치킨,전남 여수시 수정동 777-1 유탑마리나 호텔리조트,307295,5,1970-01-01 00:00:00,night
1,12356,가마골 관광농원,오리백숙,전남 담양군 용면 용연리771번지,64350,5,1970-01-01 00:00:00,night
2,28692,고서손짜장,중국요리,전남 담양군 고서면 성월리 613-11,23284,4,1970-01-01 00:00:00,night
3,45827,금강민물,참가자미,전남 화순군 화순읍 교리 237-6,23284,3,2019-04-08 15:50:43,lunch
4,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,516840,4,1970-01-01 00:00:00,night
5,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,385502,4,1970-01-01 00:00:00,night
6,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,5666,4,1970-01-01 00:00:00,night
7,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,318007,5,1970-01-01 00:00:00,night
8,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,684085,5,2018-06-08 11:53:13,lunch
9,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,10287,5,2018-07-11 18:30:35,dinner


In [52]:
for i in range(ratings.shape[0]):
    if '전라남' in ratings['address'][i].split(" ")[0]:
        Jeollanam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Jeollanam_frame = pd.DataFrame(data=Jeollanam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Jeollanam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,5923,BBQ치킨,치킨,전남 여수시 수정동 777-1 유탑마리나 호텔리조트,307295,5,1970-01-01 00:00:00,night
1,12356,가마골 관광농원,오리백숙,전남 담양군 용면 용연리771번지,64350,5,1970-01-01 00:00:00,night
2,28692,고서손짜장,중국요리,전남 담양군 고서면 성월리 613-11,23284,4,1970-01-01 00:00:00,night
3,45827,금강민물,참가자미,전남 화순군 화순읍 교리 237-6,23284,3,2019-04-08 15:50:43,lunch
4,63449,남도예담,떡갈비,전남 담양군 월산면 화방리 378-2,516840,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
1244,359447,중화루,탕수육,전라남도 목포시 상락동2가 12-7,104830,5,1970-01-01 00:00:00,night
1245,359447,중화루,탕수육,전라남도 목포시 상락동2가 12-7,942,5,1970-01-01 00:00:00,night
1246,359447,중화루,탕수육,전라남도 목포시 상락동2가 12-7,667944,3,1970-01-01 00:00:00,night
1247,359447,중화루,탕수육,전라남도 목포시 상락동2가 12-7,213172,1,1970-01-01 00:00:00,night


In [53]:
Gyeongbuk=[]
for i in range(ratings.shape[0]):
    if '경북' in ratings['address'][i].split(" ")[0]:
        Gyeongbuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gyeongbuk_frame = pd.DataFrame(data=Gyeongbuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Gyeongbuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,3200,60계 치킨,치킨,경북 구미시 봉곡동 227-12,148115,4,1970-01-01 00:00:00,night
1,14884,가장맛있는족발,족발,경북 포항시 북구 양덕동 1413,49354,4,1970-01-01 00:00:00,night
2,14884,가장맛있는족발,족발,경북 포항시 북구 양덕동 1413,881585,4,1970-01-01 00:00:00,night
3,36539,교촌면옥,아이스크림,경북 경주시 인왕동 749-1,71205,2,1970-01-01 00:00:00,night
4,49347,길성이,누룽지백숙,경북 칠곡군 석적읍 중리 648-3,804534,4,1970-01-01 00:00:00,night
5,145138,명동반점,짜장,경북 안동시 서후면 명리 417-3,127131,4,1970-01-01 00:00:00,night
6,165179,바르미,칼국수,경북 구미시 오태동 95-2,150132,4,1970-01-01 00:00:00,night
7,210016,삼복식당,파스타,경북 구미시 신평동 150-31 신평시장,148115,4,1970-01-01 00:00:00,night
8,211759,삼포식당,복지리,경북 경주시 황남동,315530,2,1970-01-01 00:00:00,night
9,228891,소우주,돈까스,경북 청도군 화양읍 유등리 1194 청도 복사꽃마을,213776,4,1970-01-01 00:00:00,night


In [54]:
for i in range(ratings.shape[0]):
    if '경상북' in ratings['address'][i].split(" ")[0]:
        Gyeongbuk.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gyeongbuk_frame = pd.DataFrame(data=Gyeongbuk, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Gyeongbuk_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,3200,60계 치킨,치킨,경북 구미시 봉곡동 227-12,148115,4,1970-01-01 00:00:00,night
1,14884,가장맛있는족발,족발,경북 포항시 북구 양덕동 1413,49354,4,1970-01-01 00:00:00,night
2,14884,가장맛있는족발,족발,경북 포항시 북구 양덕동 1413,881585,4,1970-01-01 00:00:00,night
3,36539,교촌면옥,아이스크림,경북 경주시 인왕동 749-1,71205,2,1970-01-01 00:00:00,night
4,49347,길성이,누룽지백숙,경북 칠곡군 석적읍 중리 648-3,804534,4,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
1560,359290,중앙통닭,안동찜닭,경상북도 안동시 서부동 181-2,50050,4,1970-01-01 00:00:00,night
1561,359409,중화각,백짬뽕,경상북도 포항시 남구 효자동 308-32,134049,4,1970-01-01 00:00:00,night
1562,359409,중화각,백짬뽕,경상북도 포항시 남구 효자동 308-32,31932,2,1970-01-01 00:00:00,night
1563,359658,중흥반점,닭갈비,경상북도 경주시 성건동 634-1,7623,4,2018-01-10 21:26:43,dinner


In [55]:
Gyeongnam=[]
for i in range(ratings.shape[0]):
    if '경남' in ratings['address'][i].split(" ")[0]:
        Gyeongnam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gyeongnam_frame = pd.DataFrame(data=Gyeongnam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time","day_type"))
Gyeongnam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,24492,계경목장,소고기,경남 창원시 성산구 상남동 14-7,50713,4,1970-01-01 00:00:00,night
1,27610,고메,스시,경남 진주시 가좌동 1934-1,8954,3,1970-01-01 00:00:00,night
2,31981,골목횟집,생대구탕,경남 창원시 진해구 용원동 823-5,131651,1,2017-03-27 23:44:32,night
3,32504,곱돌이,곱창전골,경남 창원시 진해구 석동 133-5,326165,4,1970-01-01 00:00:00,night
4,45947,금강산면옥,해물갈비찜,경남 창원시 진해구 충무동 13-10,135920,5,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
97,329936,이화원,밥집,경남 통영시 광도면 죽림리 1570-5,178039,4,1970-01-01 00:00:00,night
98,345101,전주콩나루콩나물국밥,콩나물국밥,경남 창원시 진해구 석동 650-17,326165,3,1970-01-01 00:00:00,night
99,345538,전통순대촌,전통순대,경남 창원시 의창구 무곡리 120-11,247994,3,2018-12-15 14:14:02,lunch
100,346320,정가불떡,불고기떡볶이,경남 창원시 마산합포구 창동 28,30101,4,2018-09-18 14:41:42,lunch


In [56]:
for i in range(ratings.shape[0]):
    if '경상남' in ratings['address'][i].split(" ")[0]:
        Gyeongnam.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Gyeongnam_frame = pd.DataFrame(data=Gyeongnam, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Gyeongnam_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,24492,계경목장,소고기,경남 창원시 성산구 상남동 14-7,50713,4,1970-01-01 00:00:00,night
1,27610,고메,스시,경남 진주시 가좌동 1934-1,8954,3,1970-01-01 00:00:00,night
2,31981,골목횟집,생대구탕,경남 창원시 진해구 용원동 823-5,131651,1,2017-03-27 23:44:32,night
3,32504,곱돌이,곱창전골,경남 창원시 진해구 석동 133-5,326165,4,1970-01-01 00:00:00,night
4,45947,금강산면옥,해물갈비찜,경남 창원시 진해구 충무동 13-10,135920,5,1970-01-01 00:00:00,night
...,...,...,...,...,...,...,...,...
2379,359196,중앙식당,두루치기,경상남도 거제시 옥포2동 529-8,365662,5,1970-01-01 00:00:00,night
2380,360465,지구당,소고기덮밥,경상남도 창원시 마산회원구 합성동 264-14,518267,5,2016-12-19 17:20:18,dinner
2381,360466,지구당,소고기덮밥,경상남도 창원시 성산구 상남동 5-3,813855,5,1970-01-01 00:00:00,night
2382,360466,지구당,소고기덮밥,경상남도 창원시 성산구 상남동 5-3,344191,4,1970-01-01 00:00:00,night


In [57]:
Jeju=[]
for i in range(ratings.shape[0]):
    if '제주' in ratings['address'][i].split(" ")[0]:
        Jeju.append(
                [ratings["store_id"][i],ratings["store_name"][i],ratings["category"][i],ratings["address"][i],ratings["user_id"][i],ratings["score"][i],ratings["reg_time"][i],ratings["day_type"][i]]
        )

Jeju_frame = pd.DataFrame(data=Jeju, columns=("store_id","store_name","category","address", "user_id", "score", "reg_time", "day_type"))
Jeju_frame

,store_id,store_name,category,address,user_id,score,reg_time,day_type
0,139,100프로코코넛카페,뒷고기,제주특별자치도 제주시 구좌읍 월정리 670-5,293161,4,2019-04-10 15:03:09,lunch
1,1098,204키친,파스타,제주특별자치도 제주시 이도2동,401127,2,1970-01-01 00:00:00,night
2,1264,24시국수회관,고기국수,제주특별자치도 제주시 삼도1동 797-8,110504,5,2018-03-16 00:08:59,night
3,1264,24시국수회관,고기국수,제주특별자치도 제주시 삼도1동 797-8,370206,3,2018-08-05 08:54:03,morning
4,1264,24시국수회관,고기국수,제주특별자치도 제주시 삼도1동 797-8,49598,5,2019-05-09 10:28:09,morning
...,...,...,...,...,...,...,...,...
3167,359190,중앙식당,성게보말국,제주 서귀포시 안덕면 화순리 1077-1,276312,4,1970-01-01 00:00:00,night
3168,359190,중앙식당,성게보말국,제주 서귀포시 안덕면 화순리 1077-1,117313,2,1970-01-01 00:00:00,night
3169,359190,중앙식당,성게보말국,제주 서귀포시 안덕면 화순리 1077-1,145101,3,1970-01-01 00:00:00,night
3170,359291,중앙통닭,마늘통닭,제주특별자치도 서귀포시 중앙동 275-27,49598,4,1970-01-01 00:00:00,night


In [58]:
35134+4084+2071+2884+1004+2324+754+277+15180+3126+1180+1662+1630+1249+1565+2384+3172

79680

In [59]:
79680 - 79752

-72

**Seoul_frame	Busan_frame	Daegu_frame	Inchen_frame	Gwangju_frame	Daejeon_frame	<br>
    Ulsan_frame	Sejong_frame Gyeonggi_frame	Gangwon_frame	Chungbuk_frame	Chungnam_frame	 <br>
    Jeollabuk_frame	Jeollanam_frame	Gyeongbuk_frame	Gyeongnam_frame	Jeju_frame**

In [61]:
Seoul_frame.to_json('data/Seoul_frame.json', orient='table')
Busan_frame.to_json('data/Busan_frame.json', orient='table')
Daegu_frame.to_json('data/Daegu_frame.json', orient='table')
Inchen_frame.to_json('data/Inchen_frame.json', orient='table')
Gwangju_frame.to_json('data/Gwangju_frame.json', orient='table')
Daejeon_frame.to_json('data/Daejeon_frame.json', orient='table')
Ulsan_frame.to_json('data/Ulsan_frame.json', orient='table')
Sejong_frame.to_json('data/Sejong_frame.json', orient='table')
Gyeonggi_frame.to_json('data/Gyeonggi_frame.json', orient='table')
Gangwon_frame.to_json('data/Gangwon_frame.json', orient='table')
Chungbuk_frame.to_json('data/Chungbuk_frame.json', orient='table')
Chungnam_frame.to_json('data/Chungnam_frame.json', orient='table')
Jeollabuk_frame.to_json('data/Jeollabuk_frame.json', orient='table')
Jeollanam_frame.to_json('data/Jeollanam_frame.json', orient='table')
Gyeongbuk_frame.to_json('data/Gyeongbuk_frame.json', orient='table')
Gyeongnam_frame.to_json('data/Gyeongnam_frame.json', orient='table')
Jeju_frame.to_json('data/Jeju_frame.json', orient='table')

# 4. 대전 지역(특정지역)에서 메뉴 추천받기

In [62]:
import json
import pandas as pd
import numpy as np
import os
import shutil
import pickle
from lightfm.data import Dataset
from scipy.io import mmwrite
from lightfm.cross_validation import random_train_test_split
from hyperopt import fmin, hp, tpe, Trials
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

In [232]:
Daejeon_frame # 2324

,store_id,store_name,category,address,user_id,score,reg_time,day_type,average_rating
0,248,111 7meal,함박스테이크,대전광역시 유성구 어은동 111-7 1층,583035,4,2019-03-01 00:24:50,night,4.000000
1,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,314095,5,1970-01-01 00:00:00,night,4.666667
2,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,328037,5,1970-01-01 00:00:00,night,4.666667
3,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,190973,4,1970-01-01 00:00:00,night,4.666667
4,737,1987루프탑,루프탑라운지,대전광역시 서구 탄방동 740,421128,4,1970-01-01 00:00:00,night,3.333333
...,...,...,...,...,...,...,...,...,...
2319,358026,죽이야기,죽집,대전광역시 서구 둔산1동 1412,151946,4,1970-01-01 00:00:00,night,4.000000
2320,358161,준구네껍데기,껍데기,대전광역시 중구 유천2동 196,96614,1,1970-01-01 00:00:00,night,1.000000
2321,358352,준호네춘천숯불닭갈비,숯불닭갈비,대전광역시 유성구 전민동 336-4,362995,3,1970-01-01 00:00:00,night,3.000000
2322,358949,중부회24시수산시장,대하구이,대전광역시 동구 홍도동 32,194919,5,1970-01-01 00:00:00,night,5.000000


In [233]:
# 대전의 모든 가게 정보
global all_stores_data
all_stores_data = Daejeon_frame
all_stores_data['average_rating'] = all_stores_data['score'].groupby(all_stores_data["store_id"]).transform('mean')

all_store_frame = pd.DataFrame(data=all_stores_data, columns=("store_id","store_name", "category", "address", "average_rating"))
all_store_frame = all_store_frame.drop_duplicates() # 중복제거 
all_store_frame

,store_id,store_name,category,address,average_rating
0,248,111 7meal,함박스테이크,대전광역시 유성구 어은동 111-7 1층,4.000000
1,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,4.666667
4,737,1987루프탑,루프탑라운지,대전광역시 서구 탄방동 740,3.333333
7,1094,2046팬스테이크,팬스테이크,대전광역시 중구 문화동 1-16,4.000000
8,1258,24시강청골순대국밥,순대국밥,대전광역시 대덕구 석봉동 406-12,4.000000
...,...,...,...,...,...
2319,358026,죽이야기,죽집,대전광역시 서구 둔산1동 1412,4.000000
2320,358161,준구네껍데기,껍데기,대전광역시 중구 유천2동 196,1.000000
2321,358352,준호네춘천숯불닭갈비,숯불닭갈비,대전광역시 유성구 전민동 336-4,3.000000
2322,358949,중부회24시수산시장,대하구이,대전광역시 동구 홍도동 32,5.000000


In [234]:
all_store_frame = all_store_frame.reset_index()
del all_store_frame['index']
all_store_frame

,store_id,store_name,category,address,average_rating
0,248,111 7meal,함박스테이크,대전광역시 유성구 어은동 111-7 1층,4.000000
1,335,123막창,막창무한리필,대전광역시 서구 둔산동 1848,4.666667
2,737,1987루프탑,루프탑라운지,대전광역시 서구 탄방동 740,3.333333
3,1094,2046팬스테이크,팬스테이크,대전광역시 중구 문화동 1-16,4.000000
4,1258,24시강청골순대국밥,순대국밥,대전광역시 대덕구 석봉동 406-12,4.000000
...,...,...,...,...,...
1291,358026,죽이야기,죽집,대전광역시 서구 둔산1동 1412,4.000000
1292,358161,준구네껍데기,껍데기,대전광역시 중구 유천2동 196,1.000000
1293,358352,준호네춘천숯불닭갈비,숯불닭갈비,대전광역시 유성구 전민동 336-4,3.000000
1294,358949,중부회24시수산시장,대하구이,대전광역시 동구 홍도동 32,5.000000


all_store_frame = all_store_frame.reset_index().rename(columns={"index": "new_store_id"})

all_store_frame

In [235]:
# 평점 데이터 희소행렬로 만듦? 
ratings_source = [(Daejeon_frame['user_id'][i], Daejeon_frame['store_id'][i]) for i in range(Daejeon_frame.shape[0])]
#ratings_source

In [236]:
# 음식점 데이터 
item_meta =all_store_frame

In [237]:
item_meta = item_meta.reset_index()
item_feature_source = [(item_meta['store_id'][i], [ item_meta['category'][i], item_meta['average_rating'][i]] ) for i in range(item_meta.shape[0]) ]
#item_feature_source

In [125]:
len(item_feature_source) # 가게 수

1296

In [126]:
len(ratings_source) # 리뷰 수

2324

In [140]:
tmp = item_meta[item_meta.columns[1:]].values.flatten()
tmp[0:10] # => [가게번호, 가게명, 카테고리, 주소, 평균평점 ]

array([248, '111 7meal', '함박스테이크', '대전광역시 유성구 어은동 111-7 1층', 4.0, 335,
       '123막창', '막창무한리필', '대전광역시 서구 둔산동 1848', 4.666666666666667],
      dtype=object)

In [142]:
item_feature_source

[(248, ['함박스테이크', 4.0]),
 (335, ['막창무한리필', 4.666666666666667]),
 (737, ['루프탑라운지', 3.3333333333333335]),
 (1094, ['팬스테이크', 4.0]),
 (1258, ['순대국밥', 4.0]),
 (1347, ['순대', 5.0]),
 (1461, ['콩나물국밥', 5.0]),
 (1465, ['콩나물국밥', 4.0]),
 (1582, ['파스타', 5.0]),
 (2278, ['삼겹살', 4.0]),
 (2505, ['닭갈비', 4.0]),
 (2511, ['닭갈비', 4.0]),
 (2946, ['피자', 3.0]),
 (3215, ['치킨', 4.0]),
 (3616, ['경양식', 4.666666666666667]),
 (4072, ['닭강정', 4.0]),
 (4652, ['브런치', 4.0]),
 (5135, ['치킨', 4.0]),
 (6225, ['치킨', 5.0]),
 (7477, ['체코', 4.0]),
 (11108, ['루프탑', 4.75]),
 (11692, ['삼겹살', 5.0]),
 (11768, ['쌀국수', 3.4]),
 (11775, ['쭈꾸미', 1.0]),
 (12558, ['곰탕', 4.0]),
 (12775, ['가마솥', 4.285714285714286]),
 (12894, ['가마솥', 3.0]),
 (13408, ['짬뽕', 4.333333333333333]),
 (13638, ['생삼겹살', 5.0]),
 (14185, ['치즈막창', 1.0]),
 (14226, ['칼국수', 3.0]),
 (14267, ['한우', 4.0]),
 (15250, ['돈가츠', 4.0]),
 (15270, ['초콜릿', 4.0]),
 (15512, ['일식', 3.0]),
 (15928, ['짬뽕', 4.0]),
 (16278, ['소갈비살', 4.0]),
 (16322, ['해물갈비찜', 2.5]),
 (16720, ['누룽지백숙', 3.0]),
 (1

In [141]:
ratings_source

[(583035, 248),
 (314095, 335),
 (328037, 335),
 (190973, 335),
 (421128, 737),
 (151946, 737),
 (43325, 737),
 (583035, 1094),
 (53470, 1258),
 (179719, 1347),
 (607732, 1461),
 (291745, 1465),
 (285715, 1465),
 (798349, 1582),
 (374341, 2278),
 (151946, 2505),
 (64397, 2505),
 (116465, 2511),
 (179719, 2946),
 (27703, 3215),
 (583035, 3616),
 (48236, 3616),
 (195748, 3616),
 (179719, 4072),
 (291745, 4652),
 (213133, 4652),
 (151946, 5135),
 (47542, 6225),
 (291745, 7477),
 (307112, 11108),
 (577925, 11108),
 (315596, 11108),
 (179719, 11108),
 (339736, 11692),
 (348417, 11768),
 (20580, 11768),
 (361993, 11768),
 (27780, 11768),
 (151946, 11768),
 (177702, 11775),
 (853410, 12558),
 (486887, 12775),
 (557138, 12775),
 (945600, 12775),
 (601714, 12775),
 (43198, 12775),
 (179719, 12775),
 (174483, 12775),
 (27780, 12894),
 (179346, 13408),
 (179719, 13408),
 (45315, 13408),
 (162976, 13638),
 (107346, 13638),
 (683346, 14185),
 (179719, 14226),
 (151946, 14267),
 (318760, 15250),
 (2

In [238]:
dataset = Dataset()
dataset.fit(users=Daejeon_frame['user_id'].unique(),
           items=Daejeon_frame['store_id'].unique(),
           item_features=item_meta[item_meta.columns[1:]].values.flatten())

interactions, weights = dataset.build_interactions(ratings_source)
item_features = dataset.build_item_features(item_feature_source)

In [231]:
item_features

<1296x5519 sparse matrix of type '<class 'numpy.float32'>'
	with 3888 stored elements in Compressed Sparse Row format>

item_features => 1296(가게수) x 4233 형태의 sparse matrix

In [239]:
# Split Train, Test data
train, test = random_train_test_split(interactions, test_percentage=0.1)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()


# Define Search Space
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)), hp.uniform('learning_rate', 0.01, 0.05)]

In [240]:
# Define Objective Function
global model
def objective(params):
    no_components, learning_rate = params
    global model
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=item_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=item_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output


In [147]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.03467, precision: 0.01898                                                                     
no_comp: 40, lrn_rate: 0.01503, precision: 0.01314                                                                     
no_comp: 30, lrn_rate: 0.03216, precision: 0.01460                                                                     
no_comp: 40, lrn_rate: 0.04758, precision: 0.01314                                                                     
no_comp: 40, lrn_rate: 0.01688, precision: 0.01314                                                                     
no_comp: 30, lrn_rate: 0.01102, precision: 0.01460                                                                     
no_comp: 10, lrn_rate: 0.02970, precision: 0.01606                                                                     
no_comp: 40, lrn_rate: 0.02162, precision: 0.01314                                                                     
no_comp: 40, lrn_rate: 0.01664, precisio

In [227]:
item_id = all_store_frame[all_store_frame['store_id']==358026].index[0]
print("아이템아이디: ",item_id)

아이템아이디:  1291


In [241]:
# Find Similar Items
def make_best_items_report(item_embeddings, store_id, num_search_items=10):
    
    item_id = all_store_frame[all_store_frame['store_id']==store_id].index[0]
    print("아이템아이디: ",item_id)
    
    # Cosine similarity(코사인 유사도 계산)
    scores = item_embeddings.dot(item_embeddings[item_id])  # (10000, )
    item_norms = np.linalg.norm(item_embeddings, axis=1)    # (10000, )
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[item_id]),
                                        key=lambda x: -x[1])
    print(similar_item_id_and_scores)
    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame(columns=['store_id','category','score'])
    
    for similar_item_id, score in similar_item_id_and_scores:
        
        store_id = all_stores_data.iloc[similar_item_id:similar_item_id+1,]
        
        print(store_id)
        print(all_stores_data[all_stores_data['store_id']==store_id].values)
#         category = Daejeon_frame[Daejeon_frame['store_id']==store_id].values[0][2]
        
        # row = pd.Series([store_id, store_name, category,score], index=best_items.columns)
        row = pd.Series([store_id, "임시카테고리",score], index=best_items.columns)
        best_items = best_items.append(row, ignore_index=True)

    return best_items

In [248]:
# Find Similar Items
def make_best_items_report(item_embeddings, src_store_id, num_search_items=10):
    src_idx = all_store_frame[all_store_frame['store_id']==src_store_id].index[0]

    # Cosine similarity
    scores = item_embeddings.dot(item_embeddings[src_idx])  # (10000, )
    item_norms = np.linalg.norm(item_embeddings, axis=1)    # (10000, )
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[src_idx]),
                                        key=lambda x: -x[1])
    print(similar_item_id_and_scores)
    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame(columns=['store_id','category','score'])
    global all_stores_data
    for tar_idx, score in similar_item_id_and_scores:
        # all_store_frame.iloc[4:5,].values
        tar_store_id = all_store_frame.iloc[tar_idx,0]
        
        print("인덱스: ", tar_idx, ", store id: ", tar_store_id)
        print(all_stores_data[all_stores_data['store_id']==tar_store_id].values)
        category = all_stores_data[all_stores_data['store_id']==tar_store_id].values[0][2]
        
        # row = pd.Series([store_id, store_name, category,score], index=best_items.columns)
        row = pd.Series([tar_store_id,category,score], index=best_items.columns)
        best_items = best_items.append(row, ignore_index=True)

    return best_items

In [243]:
item_biases, item_embeddings = model.get_item_representations(features=item_features)

In [201]:
all_store_frame.iloc[1258:1260,]

,store_id,store_name,category,address,average_rating
1258,344628,전주소머리국밥,소머리국밥,대전시 유성구 장대동 284-3,3.0
1259,345537,전통순대국밥,전통순대,대전광역시 유성구 어은동 106-1,5.0


In [252]:
report01 = make_best_items_report(item_embeddings, 1258, 10) # 순대국밥 

print("1258번 가게(순대국밥)과 유사한 음식점! ")
report01

[(4, 0.9999998), (714, 0.99147654), (149, 0.9912334), (1155, 0.99103594), (926, 0.990948), (1225, 0.9896354), (808, 0.98962086), (651, 0.98882973), (233, 0.98827726), (345, 0.98795503)]
인덱스:  4 , store id:  1258
[[1258 '24시강청골순대국밥' '순대국밥' '대전광역시 대덕구 석봉동 406-12' 53470 4
  Timestamp('2018-06-09 08:17:18') 'morning' 4.0]]
인덱스:  714 , store id:  202675
[[202675 '쁘리모' '피자무한리필' '대전광역시 유성구 봉명동 617-4' 635996 4
  Timestamp('2017-12-11 15:30:25') 'lunch' 4.0]]
인덱스:  149 , store id:  43588
[[43588 '귀선' '꼬치구이' '대전광역시 서구 둔산1동 1426' 213133 4
  Timestamp('1970-01-01 00:00:00') 'night' 4.0]]
인덱스:  1155 , store id:  316760
[[316760 '유성순대' '순대국밥' '대전광역시 중구 석교동 57-12' 205503 5
  Timestamp('2018-05-30 04:35:28') 'night' 4.0]
 [316760 '유성순대' '순대국밥' '대전광역시 중구 석교동 57-12' 362995 3
  Timestamp('2019-05-24 22:36:10') 'dinner' 4.0]]
인덱스:  926 , store id:  256475
[[256475 '쌍둥이네뽕잎해물칼국수' '수제타르트' '대전광역시 유성구 하기동 537-8' 656515 4
  Timestamp('2016-12-05 23:34:54') 'night' 4.0]]
인덱스:  1225 , store id:  331466
[[331466 '

,store_id,category,score
0,1258,순대국밥,1.000000
1,202675,피자무한리필,0.991477
2,43588,꼬치구이,0.991233
3,316760,순대국밥,0.991036
4,256475,수제타르트,0.990948
5,331466,뼈다귀,0.989635
6,228039,소바,0.989621
7,181646,탕수육,0.988830
8,71548,돼지국밥,0.988277
9,98753,제주산돼지고기,0.987955


In [251]:
report02 = make_best_items_report(item_embeddings, 2278, 10) # 삼겹살 
print("2278번가게(삼겹살) 과 유사한 음식점! ")
report02

[(9, 1.0000001), (1244, 0.99861956), (1075, 0.9952272), (1157, 0.99464506), (66, 0.9945479), (1167, 0.9942386), (390, 0.99362046), (445, 0.99314964), (161, 0.99298954), (1285, 0.9929676)]
인덱스:  9 , store id:  2278
[[2278 '3돈한마리' '삼겹살' '대전광역시 대덕구 중리동 165-20' 374341 4
  Timestamp('1970-01-01 00:00:00') 'night' 4.0]]
인덱스:  1244 , store id:  336903
[[336903 '작품식당' '삼겹살' '대전광역시 유성구 봉명동 544-7' 179719 4
  Timestamp('2018-06-18 18:26:50') 'dinner' 4.0]]
인덱스:  1075 , store id:  297319
[[297319 '온더스킬렛' '삼겹살' '대전광역시 서구 갈마동 828 1층' 583035 4
  Timestamp('2018-01-17 08:25:01') 'morning' 4.0]]
인덱스:  1157 , store id:  316799
[[316799 '유성주먹구이' '삼겹살' '대전광역시 유성구 봉명동 611-2' 179719 4
  Timestamp('2019-01-14 17:21:02') 'dinner' 4.0]]
인덱스:  66 , store id:  23747
[[23747 '경성일번가' '삼겹살' '대전광역시 서구 둔산동 1074' 190145 4
  Timestamp('1970-01-01 00:00:00') 'night' 4.0]]
인덱스:  1167 , store id:  319151
[[319151 '육하원칙' '삼겹살' '대전광역시 유성구 반석동 203-2 1층' 374341 4
  Timestamp('1970-01-01 00:00:00') 'night' 4.0]]
인덱스:  390 , st

,store_id,category,score
0,2278,삼겹살,1.000000
1,336903,삼겹살,0.998620
2,297319,삼겹살,0.995227
3,316799,삼겹살,0.994645
4,23747,삼겹살,0.994548
5,319151,삼겹살,0.994239
6,109938,오리누룽지백숙,0.993620
7,126576,족발,0.993150
8,47921,순대국,0.992990
9,354179,족발,0.992968


In [258]:
all_store_frame[all_store_frame['store_id'].isin(report02['store_id'])]


,store_id,store_name,category,address,average_rating
9,2278,3돈한마리,삼겹살,대전광역시 대덕구 중리동 165-20,4.0
66,23747,경성일번가,삼겹살,대전광역시 서구 둔산동 1074,4.0
161,47921,금화순대,순대국,대전광역시 서구 월평동 1473,4.0
390,109938,드리오리,오리누룽지백숙,대전광역시 서구 갈마1동 427-61,4.0
445,126576,마녀족발,족발,대전광역시 유성구 봉명동 666-1,4.0
1075,297319,온더스킬렛,삼겹살,대전광역시 서구 갈마동 828 1층,4.0
1157,316799,유성주먹구이,삼겹살,대전광역시 유성구 봉명동 611-2,4.0
1167,319151,육하원칙,삼겹살,대전광역시 유성구 반석동 203-2 1층,4.0
1244,336903,작품식당,삼겹살,대전광역시 유성구 봉명동 544-7,4.0
1285,354179,족발제작소,족발,대전광역시 서구 가수원동 915,4.0


In [ ]:
# 코사인 유사도로  하면 안될꺼 같다... 

# 책추천할 때 ---> 코사인유사도를 씀.
# 